# 2.2_Advanced Python

## *2.2a Object Orientied Programming with Python*

* Classes have some special methods in Python.
* `__init__` is called on the instance right after instantiation (i.e. constructor)
* `__repr__` is called by the built-in `repr()` to get a string representation, or by `str()` if there is no `__str__` method
* `__call__` is called when the call syntax `()` is used on an instance of the class

In [1]:
class Person: 
    
    def __init__(self, first_name, last_name): # constructor
        
        self.first_name = first_name
        self.last_name = last_name
        
    def __str__(self): # object representation
        return f'Person: {self.first_name} {self.last_name}'
    
    def __repr__(self): # object representation
        return f'Person({self.first_name}, {self.last_name})'
    
    def __call__(self):
        #function to be called by default on object call
        return self.talk()
        #print(super())
    
    def talk(self): # method
        return f'Hello. My name is {self.first_name} {self.last_name}. '

In [2]:
me = Person('Monty', 'Python')

print(me) # print uses __str__
# return values use __repr__
me, repr(me), str(me)

Person: Monty Python


(Person(Monty, Python), 'Person(Monty, Python)', 'Person: Monty Python')

In [3]:
# Calling the instance
me() 

'Hello. My name is Monty Python. '

## *2.2b Child class*

In [4]:
class Student(Person): # Child class
    
    def __init__(self, first_name, last_name, mat_number, university):
        
        Person.__init__(self, first_name, last_name) # Parent constructor
        self.mat_number = mat_number
        self.university = university
        self.modules = []
        self.credits = 0 # ECTS
        self.grades = [] # [3, 4, 1, 2, 2]
        
    def __repr__(self):
        info = f'{self.first_name} {self.last_name}\nStudent: {self.university} {self.mat_number}'
        
        if len(self.modules) > 0:      
            classes = ', '.join(self.modules)
            info += f'\nCredits: {self.credits} ECTS in {classes} ' + \
                f'avg score: {sum(self.grades)/len(self.grades):0.1f}'
        return  info
        
    def talk(self):
        # parent's method call
        return (
            super().talk()
            + f"I'm studying at {self.university}. "
            + f"My matriculation number is {self.mat_number}. "
        )
    
    def exam(self, module_name: str, credit: int, grade: float) -> None:
        """
            Adds exam info and credits 
        """
        self.modules += [module_name]
        self.credits += credit
        self.grades += [grade]

## *2.2c Student class usage*

In [5]:
#object creation
anna = Student('Anna', 'Mustermann', 4345325, 'TU Berlin')

# function talk is invoked
anna.talk() # or just anna()

"Hello. My name is Anna Mustermann. I'm studying at TU Berlin. My matriculation number is 4345325. "

In [6]:
anna.exam('CS', 6, 1.7)
anna.exam('BIO', 12, 2.7)

#print(anna)
#print(repr(anna))
anna

Anna Mustermann
Student: TU Berlin 4345325
Credits: 18 ECTS in CS, BIO avg score: 2.2

## *2.2d Multiple class inheritance*

In [7]:
class Employee:
    
    def __init__(self, company, position, salary):
        
        self.company = company
        self.position = position
        self.salary = salary
        
    def __repr__(self):
        return f"Employee: {self.company} as {self.position}"
    
    def talk(self):
        return (
            super().talk()
            + f"I'm employed at {self.company} as {self.position}. "
        )

In [8]:
class HiWi(Student, Employee):

    def __init__(self, first_name, last_name, mat_number, university, salary):
        
        Student.__init__(self, first_name, last_name, mat_number, university)
        Employee.__init__(self, university, 'HiWi', salary)
        
    def __repr__(self):
        
        info = Student.__repr__(self) # Note: super() first inherited class is beeing called 
        info += f'\nPosition: {self.position} with salary {self.salary}$'
        return info + f'\n{Employee.__repr__(self)}'

In [9]:
me = HiWi('Monty','Python', 123456, 'TU Berlin', 1000)
me.exam('BIO', 6, 2.7)
me.exam('PyML', 3, 1.7)
me

Monty Python
Student: TU Berlin 123456
Credits: 9 ECTS in BIO, PyML avg score: 2.2
Position: HiWi with salary 1000$
Employee: TU Berlin as HiWi

## *2.2e Method Resolution Order*

In [10]:
# Classes with multiply inherited classes have a Method Resolution Order (MRO), which is a flat ordering of which parent class' methods/attributes will be accesses. 
# This can be investigated with the .mro() method

In [11]:
class Base:
    def __call__(self):
        return ''

class A(Base):
    num = 1
    def __call__(self):
        return 'A' + super().__call__()

class B(Base):
    num = 2
    def __call__(self):
        return 'B' + super().__call__()

class C(Base):
    num = 3
    def __call__(self):
        return 'C' + super().__call__()


class D(A, B, C):
    pass

print(D()()) #D() ist for instanzieren und zweites () um aufzurufen?
print(D.num)
print(D.mro())

ABC
1
[<class '__main__.D'>, <class '__main__.A'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.Base'>, <class 'object'>]


In [12]:
# The type built-in can also create new types on the fly

print(type('E', (A, B, C), {})()())
print(type('F', (B, C, A), {})()())
print(type('G', (B, A, C), {})()())

ABC
BCA
BAC


## *2.2f Staticmethod, Classmethod*

In [13]:
# We can specify static methods and class-methods with special built-in decorators

class Wow:
    def instance_method(self):
        return type(self)
    
    @classmethod
    def class_method(cls):
        return type(cls)
    
    @staticmethod
    def static_method():
        return 'No arguments here!'
    
print(Wow.static_method())
print(Wow.class_method())
print(Wow().instance_method())
print(Wow().class_method())

# Types have the type type in Python

No arguments here!
<class 'type'>
<class '__main__.Wow'>
<class 'type'>


## *2.2g Attributes and Descriptors*

In [14]:
# All objects have attributes in Python, they may be accessed with the . syntax.

class MyClass:
    var = 3
    def __init__(self):
        self.num = 2
    def func(self):
        return self.num + self.var
    
print(MyClass.var)

3


In [15]:
MyClass.obj = 4
print(MyClass.obj)
del MyClass.obj

4


In [16]:
# We can also access, set or delete attributes with the setattr, getattr and delattr built-ins.

print(getattr(MyClass, 'thing', 'This is a default value!'))
setattr(MyClass, 'other', 15)
print(MyClass.other)
delattr(MyClass, 'other')

This is a default value!
15


In [17]:
# We can check wether an attribute exists using hasattr, and list all attributes using dir
print(hasattr(MyClass, 'var'))
print(dir(MyClass))

True
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'func', 'var']


## *2.2h Special Methods: Operators*

In [18]:
class Dough:

    def __init__(self, weight=0.5, volume=0.5):
        self.weight = weight
        self.volume = volume
        
    def __repr__(self):
        attributes = ', '.join(f'{key}={getattr(self, key)}' for key in dir(self) if key[0] != '_')
        return f'{type(self).__name__}({attributes})'
    
    def __add__(self, other):
        return type(self)(self.weight + other.weight, self.volume + other.volume)
    
    def __iadd__(self, other):
        self.weight += other.weight
        self.volume += other.volume
        return self
    
    def __mul__(self, other):
        return type(self)(self.weight * other)
    
    def __rmul__(self, other):
        return self * other

In [19]:
mydough = Dough()
print(mydough)
mydough += Dough(5, 5)
print(mydough)

Dough(3, 4), Dough(1) + Dough(), 3 * Dough(), Dough() * 3

Dough(volume=0.5, weight=0.5)
Dough(volume=5.5, weight=5.5)


(Dough(volume=4, weight=3),
 Dough(volume=1.0, weight=1.5),
 Dough(volume=0.5, weight=1.5),
 Dough(volume=0.5, weight=1.5))

## *2.2i Special Methods: Container*

In [20]:
class Bucket:
    def __init__(self):
        self.keys = []
        self.values = []
        
    def __repr__(self):
        attributes = ', '.join(f'{key}={getattr(self, key)}' for key in dir(self) if key[0] != '_')
        return f'{type(self).__name__}({attributes})'
    
    def __getitem__(self, key):
        return self.values[self.keys.index(key)]
    
    def __setitem__(self, key, value):
        try:
            index = self.keys.index(key)
        except ValueError:
            self.keys.append(key)
            self.values.append(value)
        else:
            self.values[index] = value
    
    def __delitem__(self, key):
        index = self.keys.index(key)
        del self.keys[index]
        del self.values[index]
    
    def __len__(self):
        return sum(1 for _ in self.keys)
    
    def __contains__(self, key):
        return key in self.keys
    
    def __iter__(self):
        return iter(self.keys)

In [21]:
bucket = Bucket()
for i in range(5):
    bucket[f'{i:03d}'] = i * 'a'

print(bucket['002'])
del bucket['003']
print(len(bucket))
list(bucket)
bucket

aa
4


Bucket(keys=['000', '001', '002', '004'], values=['', 'a', 'aa', 'aaaa'])

## *2.2j Context Manager Types*

In [22]:
class CommitBucket(Bucket):
    def __init__(self, base):
        super().__init__()
        for key, val in base.items():
            self[key] = val

    def __enter__(self):
        self._temp_bucket = Bucket()
        return self._temp_bucket
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        for key in self._temp_bucket:
            self[key] = self._temp_bucket[key]
        del self._temp_bucket

In [23]:
mybucket = CommitBucket({'a': 1, 'b':2})

with mybucket as temp_bucket:
    temp_bucket['c'] = 3
    print(mybucket)
    print(temp_bucket)
print(mybucket)

CommitBucket(keys=['a', 'b'], values=[1, 2])
Bucket(keys=['c'], values=[3])
CommitBucket(keys=['a', 'b', 'c'], values=[1, 2, 3])


## *2.2k Generators*

In [24]:
# Generators are special functions which execution you can stop and rerun.

# Infinite counter as a generator
def counter():
    print('Conter initialized')
    
    n = 0
    while True:
        yield n # similar to return
        n += 1

In [25]:
# generator type is returned
counter()

<generator object counter at 0x0000022613FC2650>

In [26]:
# The next built-in takes one step of the generator and returns the current element

cnt_gen = counter()
next(cnt_gen)

Conter initialized


0

In [27]:
next(cnt_gen)

1

In [28]:
# Generator objects behave like iterators
for i in counter():
    if i < 5:
        print(i)
    else:
        break

Conter initialized
0
1
2
3
4


## *2.2l Generators expressions*

In [29]:
gen_obj = (x**2 for x in range(100_000) if x % 10 == 0)
gen_obj

<generator object <genexpr> at 0x0000022613FC2B90>

In [30]:
next(gen_obj), next(gen_obj), next(gen_obj)

(0, 100, 400)

In [31]:
gen_list = [x**2 for x in range(100_000) if x % 10 == 0]

In [32]:
sys.getsizeof(gen_obj), sys.getsizeof(gen_list)

NameError: name 'sys' is not defined

## *2.2m Decorators*

In [33]:
# Define a decorator which wrapps a custom function

def benchmark(func):
    
    from time import time #import time to get current time
    
    def wrapper(*args, **kwargs):
      
        start = time() # start measuring time second passed since begin of UNIX time 1970 sthm
        res = func(*args, **kwargs)
        end = time() # end measuring time
        
        ms = (end - start) * 1000
        print(f"Elapsed time: {ms:0.6f} ms")

        return res
    
    return wrapper

In [34]:
@benchmark
def sum_up(n, step=1):
    cnt = 0
    for i in range(n):
        if i % step == 0:
            cnt += i
    return cnt 

In [35]:
#execture the fn
res = sum_up(10_000, step = 5)
print(res)

Elapsed time: 0.986814 ms
9995000


## *2.2n Caching*

In [36]:
'''
If a function is being executed many times and 
   -it takes a long time to return the results,
   - it produces the same results for the same inputs.

Then then we might cache the results to improve the performance.
'''

'\nIf a function is being executed many times and \n   -it takes a long time to return the results,\n   - it produces the same results for the same inputs.\n\nThen then we might cache the results to improve the performance.\n'

In [37]:
from functools import lru_cache

@lru_cache(10) # number of func returns to cache
def sum_up(n, step=1):
    cnt = 0
    for i in range(n):
        if i % step == 0:
            cnt += i
    return cnt 

In [38]:
@benchmark
def run(*args, **kwargs):
    res = sum_up(*args, **kwargs)
    print(sum_up.cache_info())
    return res

## *2.2o Running the cached function*

In [39]:
run(10_000, step=20)

CacheInfo(hits=0, misses=1, maxsize=10, currsize=1)
Elapsed time: 3.002405 ms


2495000

In [40]:
run(10_000, step=25)

CacheInfo(hits=0, misses=2, maxsize=10, currsize=2)
Elapsed time: 0.994921 ms


1995000

## *2.2p Combinatorics*

In [41]:
# tertools is a built-in packages that provides a lot of useful functionality for iterators

import itertools as it

myset = {1, 2, 3, 4}

list(it.islice(myset, 1, 3))

[2, 3]

In [42]:
# All unique combinations

lst = [1, 0, 2]

[i for i in it.combinations(lst, r=2)]

[(1, 0), (1, 2), (0, 2)]

In [43]:
# All possible combinations 
[i for i in it.combinations_with_replacement(lst, r=3)]

[(1, 1, 1),
 (1, 1, 0),
 (1, 1, 2),
 (1, 0, 0),
 (1, 0, 2),
 (1, 2, 2),
 (0, 0, 0),
 (0, 0, 2),
 (0, 2, 2),
 (2, 2, 2)]

In [44]:
# All permutations
[i for i in it.permutations(lst)]

[(1, 0, 2), (1, 2, 0), (0, 1, 2), (0, 2, 1), (2, 1, 0), (2, 0, 1)]

In [45]:
# Cartesian product A x B
[i for i in it.product(lst, lst)]

[(1, 1), (1, 0), (1, 2), (0, 1), (0, 0), (0, 2), (2, 1), (2, 0), (2, 2)]

## *2.2 Scikit-learn package*

## *2.2q Standard Scaler*

In [46]:
import numpy as np

from sklearn.preprocessing import StandardScaler

In [47]:
x = np.array([0, 0, 0, 0, 1, 1, 1, 2, 2, 2])

scaler = StandardScaler(with_mean=True, with_std=True)
x = x.reshape(-1, 1) # x is not a col-vector
print(x.shape)

scaler.fit(x); # train

(10, 1)


In [48]:
# mu
print(scaler.mean_)

# sigma
print(scaler.scale_)

[0.9]
[0.83066239]


In [49]:
z = scaler.transform(x)
z, z.mean(), z.std()

(array([[-1.08347268],
        [-1.08347268],
        [-1.08347268],
        [-1.08347268],
        [ 0.12038585],
        [ 0.12038585],
        [ 0.12038585],
        [ 1.32424438],
        [ 1.32424438],
        [ 1.32424438]]),
 0.0,
 1.0)

## *2.2r Min Max Scaler*

In [50]:
from sklearn.preprocessing import MinMaxScaler

feature_range = (-10, 10)
min_max_scaler = MinMaxScaler(feature_range, clip=False) # outside ranges are allowed
#MinMaxScaler?

In [51]:
x = np.array([0, 0, 0, 0, 1, 1, 1, 2, 2, 2])[:, None] # added an axis to be a col-vector
print(x.shape)

min_max_scaler.fit(x); # return self__repr__()

(10, 1)


In [52]:
x = np.array([-10, 0, 0, 0, 1, 1, 1, 2, 2, 2])[:, None]

min_max_scaler.transform(x).squeeze()

array([-110.,  -10.,  -10.,  -10.,    0.,    0.,    0.,   10.,   10.,
         10.])